# **Dans cette notebook, on va réaliser un prototype de chatbot, qui permettra de naviguer dans les données fluidement et plus simplement**

In [ ]:
pip install langchain openai faiss-cpu tiktoken pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 21.0 MB/s eta 0:00:00


In [ ]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.5 MB/s eta 0:00:00


## 1.   Chargement des données json






In [ ]:
import json
from langchain.schema import Document

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_sample = '/content/drive/MyDrive/Colab Notebooks/customer_feedback_sample.json'
# Charger les données JSON
with open(file_sample, "r", encoding="utf-8") as f:
    raw_data = json.load(f)

## 2.  Convertir en document pour LangChain

In [ ]:
# Convertir chaque entrée en Document
documents = []
for entry in raw_data:
    content = f"""
    Date: {entry["submission_date"]}
    Channel: {entry["channel"]}
    Service Type: {entry["service_type"]}
    Agent: {entry["agent_name"]}
    Name: {entry["name"]}, Age: {entry["age"]}, City: {entry["city"]}
    Rating: {entry["quality_rating"]}
    Recommend: {entry["recommend"]}
    Comments: {entry["comments"]}
    Tags: {', '.join(entry['tags'])}
    """
    documents.append(Document(page_content=content, metadata={"agent_name": entry["agent_name"]}))

## 3.  Créer une base vectorielle avec des embeddings

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(documents, embedding_model)

# Sauvegarder pour usage ultérieur
db.save_local("feedback_faiss_index")

/tmp/ipython-input-3461259160.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


## 4.  Charger un modèle Mistral via HuggingFace Hub

Le modèle chargé depuis HuggingFace ici est **Mistral-7B-instruct**, qui necessite un GPU ou un CPU assez puissant

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", torch_dtype="auto")


pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    do_sample=True,
    return_full_text=False,  # Critical: only return new tokens
    pad_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-2956988059.py:24: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)




## 5.   Création de la chaine RAG avec RetrievalQA



In [ ]:
from langchain.prompts import PromptTemplate

template = """<s>[INST] Use the following pieces of context to answer the question at the end.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    And also provide the answer in the same language as the one of the question.

Context:
{context}

Question: {question} [/INST]

Helpful Answer:"""
CUSTOM_PROMPT = PromptTemplate(
    template=template, input_variables=["context", "question"]
)

In [ ]:
from langchain.chains import RetrievalQA

# Charger la base vectorielle
db = FAISS.load_local("feedback_faiss_index", embedding_model, allow_dangerous_deserialization=True)

retriever = db.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": CUSTOM_PROMPT}
)

## 6.  Test de quelques requêtes

In [ ]:
query = "Combien de personnes on répondu à ce formulaire?"
response = qa_chain.invoke({"query": query})
print(response['result'])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




The data provided shows that five different people have responded to this form.


In [ ]:
%pip install --upgrade --quiet  langchain-community